In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from pUtilities import show_2D_array
from pSTIR import *

data_path = petmr_data_path('pet') + '/mMR'
#data_path='/home/sirfuser/data/NEMA'
print('Finding files in %s' % data_path)
os.chdir(data_path)

In [ ]:
# check content of current directory using an iPython "magic" command
%ls

In [ ]:
# set filenames 
# input files
list_file = '20170809_NEMA_60min_UCL.l.hdr';
norm_file = 'norm.n.hdr'
attn_file = 'mu_map.hv'
# output filename prefixes
sino_file = 'sino'

In [ ]:
# redirect STIR messages to some files
# you can check these if things go wrong
msg_red = MessageRedirector('info.txt', 'warn.txt')

In [ ]:
template_acq_data = AcquisitionData('Siemens_mMR',span=11, max_ring_diff=15,view_mash_factor=2)
template_acq_data.write('template.hs')

In [ ]:
# create listmode-to-sinograms converter object
lm2sino = ListmodeToSinograms()

# set input, output and template files
lm2sino.set_input(list_file)
lm2sino.set_output_prefix(sino_file)
lm2sino.set_template('template.hs')

In [ ]:
# set interval
lm2sino.set_time_interval(0,10)
# set up the converter
lm2sino.set_up()
# convert
lm2sino.process()

In [ ]:
%ls

In [ ]:
# get access to the sinograms
acq_data = lm2sino.get_output()
# copy the acquisition data into a Python array
acq_array = acq_data.as_array()
print('acquisition data dimensions: %dx%dx%d' % acq_array.shape)
# use a slice number for display that is appropriate for the NEMA phantom
z = 71
show_2D_array('Acquisition data', acq_array[z,:,:])

In [ ]:
# select acquisition model that implements the geometric
# forward projection by a ray tracing matrix multiplication
acq_model = AcquisitionModelUsingRayTracingMatrix();
acq_model.set_num_tangential_LORs(10);

In [ ]:
# define objective function to be maximized as
# Poisson logarithmic likelihood (with linear model for mean)
obj_fun = make_Poisson_loglikelihood(acq_data)
obj_fun.set_acquisition_model(acq_model)

In [ ]:
# select Ordered Subsets Maximum A-Posteriori One Step Late as the
# reconstruction algorithm (since we are not using a penalty, or prior, in
# this example, we actually run OSEM);
# this algorithm does not converge to the maximum of the objective function
# but is used in practice to speed-up calculations
# See the reconstruction demos for more complicated examples
recon = OSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)
                             
num_subsets = 7
num_subiterations = 4
recon.set_num_subsets(num_subsets)
recon.set_num_subiterations(num_subiterations)

In [ ]:
# create initial image estimate of dimensions and voxel sizes
# compatible with the scanner geometry (included in the AcquisitionData
# object acq_data) and initialize each voxel to 1.0
nxny = (127, 127)
initial_image = acq_data.create_uniform_image(1.0, nxny)

In [ ]:
image=initial_image
# set up the reconstructor based on a sample image
# (checks the validity of parameters, sets up objective function
# and other objects involved in the reconstruction, which involves
# computing/reading sensitivity image etc etc.)
recon.set_up(image)

# set the initial image estimate
recon.set_current_estimate(image)

# reconstruct
recon.process()

# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

In [ ]:
asm_norm = AcquisitionSensitivityModel(norm_file)

In [ ]:
acq_model.set_acquisition_sensitivity(asm_norm)

In [ ]:
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
image=initial_image
# set up the reconstructor based on a sample image
# (checks the validity of parameters, sets up objective function
# and other objects involved in the reconstruction, which involves
# computing/reading sensitivity image etc etc.)
recon.set_up(image)

# set the initial image estimate
recon.set_current_estimate(image)

# reconstruct
recon.process()

# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

In [ ]:
# read attenuation image
attn_image = ImageData(attn_file)
z = 71
attn_image.show(z)

In [ ]:
attn_acq_model = AcquisitionModelUsingRayTracingMatrix()
attn_acq_model.set_num_tangential_LORs(10)
asm_attn = AcquisitionSensitivityModel(attn_image, attn_acq_model)
# temporary fix pending attenuation offset fix in STIR:
# converting attenuation into 'bin efficiency'
asm_attn.set_up(acq_data)
attn_factors = AcquisitionData(acq_data)
attn_factors.fill(1.0)
print('applying attenuation (please wait, may take a while)...')
asm_attn.unnormalise(attn_factors)

In [ ]:
asm_attn = AcquisitionSensitivityModel(attn_factors)

In [ ]:
# chain attenuation and ECAT8 normalisation
asm = AcquisitionSensitivityModel(asm_norm, asm_attn)

acq_model.set_acquisition_sensitivity(asm)
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
image=initial_image
# set up the reconstructor based on a sample image
# (checks the validity of parameters, sets up objective function
# and other objects involved in the reconstruction, which involves
# computing/reading sensitivity image etc etc.)
recon.set_up(image)

# set the initial image estimate
recon.set_current_estimate(image)

# reconstruct
recon.process()

# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

In [ ]:
# set interval
lm2sino.set_time_interval(0,.002)
# set up the converter
lm2sino.set_up()

In [ ]:
# Get the randoms
randoms = lm2sino.estimate_randoms()

In [ ]:
randoms_array=randoms.as_array()
show_2D_array('randoms',randoms_array[z,:,:])

In [ ]:
acq_model.set_background_term(randoms)
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
image=initial_image
# set up the reconstructor based on a sample image
# (checks the validity of parameters, sets up objective function
# and other objects involved in the reconstruction, which involves
# computing/reading sensitivity image etc etc.)
recon.set_up(image)

In [ ]:
# set the initial image estimate
recon.set_current_estimate(image)

# reconstruct
recon.process()

# show reconstructed image
image_array = recon.get_current_estimate().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

In [ ]:
show_3D_array(image_array[(z,z+15),:,:])